In [14]:
import numpy as np

def gen_data(n, d, p):
    # true ATE is 1
    W = np.random.normal(0, 1, size=(n, d))
    D = np.random.binomial(1, p, size=(n,))
    Y = (1 + 2 * W[:, 0]) * D - W[:, 0] + np.random.normal(size=(n,))
    return 1 + W, D, Y

# Low dimensional covariates

In [31]:
n = 100
d = 3
p = .2
W, D, Y = gen_data(n, d, p)

In [32]:
import statsmodels.api as sm

sm.OLS(Y, sm.add_constant(D)).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,-0.0232,0.157,-0.148,0.883,-0.331,0.285
x1,1.4266,0.368,3.875,0.000,0.705,2.148


In [33]:
W = W - W.mean(axis=0)
sm.OLS(Y, np.hstack([sm.add_constant(D), W])).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,-0.0425,0.127,-0.336,0.737,-0.291,0.206
x1,1.5146,0.430,3.523,0.000,0.672,2.357
x2,-0.4565,0.203,-2.250,0.024,-0.854,-0.059
x3,0.0228,0.156,0.146,0.884,-0.283,0.328
x4,0.0422,0.132,0.320,0.749,-0.216,0.301


In [34]:
W = W - W.mean(axis=0)
sm.OLS(Y, np.hstack([sm.add_constant(D), W, D.reshape(-1, 1) * W])).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,-0.0491,0.112,-0.437,0.662,-0.269,0.171
x1,1.1465,0.250,4.582,0.000,0.656,1.637
x2,-0.9892,0.104,-9.513,0.000,-1.193,-0.785
x3,-0.0374,0.124,-0.303,0.762,-0.280,0.205
x4,-0.0496,0.109,-0.455,0.649,-0.263,0.164
x5,2.2830,0.188,12.152,0.000,1.915,2.651
x6,0.3946,0.249,1.585,0.113,-0.093,0.883
x7,-0.2437,0.254,-0.961,0.337,-0.741,0.253


# What if we don't demean the covariates

In [35]:
n = 100
d = 1
p = .2
W, D, Y = gen_data(n, d, p)

The non-interactive approach still converges to the correct ATE (albeit could still have higher standard error than the two means).

The characterization of the coefficient associated with the treatment is the residual-on-residual regression coefficient:
$$a=\frac{E[\tilde{Y} \tilde{D}]}{E[\tilde{D}^2]}$$
The because $W$ is independent of $D$, the BLP of $D$ using $(1,W)$ is $E[D]$ (even if $W$ is not de-meaned).

The normal equation associated with the intercept 1 gives:
$$E[(D - c - \gamma W) ] = 0 => E[D] = c + \gamma E[W]$$
Then the normal equation associated with $W$ gives:
$$E[(D - E[D] - \gamma (W - E[W])) W ] = 0 \implies \gamma = \frac{E[(D - E[D]) (W - E[W])]}{E[W(W-E[W])]} = \frac{E[D - E[D]] E[W-E[W]]}{E[W (W-E[W])] = 0}=0$$

Thus the coefficient $\alpha$ associated with $D$ in the OLS $Y\sim D, 1, W$, even if we do not de-mean $W$ is:
$$\alpha = \frac{E[(Y - \beta_0 - \beta'W)(D - E[D])]}{E[\tilde{D}^2]}$$
where $\beta_0,\beta$ is the BLP of $Y$ using $(1, W)$. Since $W$ is independent of $D$, we have $E[(\beta_0 + \beta'W)(D - E[D])]=0$ and:
$$\alpha = \frac{E[(Y - \beta_0 - \beta'W)(D - E[D])]}{E[\tilde{D}^2]} = \frac{E[Y \tilde{D}]}{E[\tilde{D}^2]}$$

Moreover, this coefficient is exactly the same as the coefficient of $Y$ using $(D, 1)$, which we have already argued that it is equal to the ATE. More concretely, the coefficient of $D$ in the BLP of $Y$ using $(D,1)$ is:
$$\alpha = \frac{E[(Y-\alpha_0) \tilde{D}]}{E[\tilde{D}^2]}$$
where $\alpha_0$ is the BLP of $Y$ using only $1$ (i.e. the mean of $Y$). But then $E[\alpha_0 \tilde{D}] = 0$. Thus we can also write:
$$\alpha = \frac{E[Y \tilde{D}]}{E[\tilde{D}^2]}$$


In [36]:
sm.OLS(Y, np.hstack([sm.add_constant(D), W])).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,0.4764,0.193,2.466,0.014,0.098,0.855
x1,1.5791,0.480,3.291,0.001,0.639,2.519
x2,-0.7412,0.124,-5.957,0.000,-0.985,-0.497


However, the interactive method does not recover the correct ATE if we don't demean the covariates (i.e. the coefficient associated with the treatment is not the ATE)

In [37]:
sm.OLS(Y, np.hstack([sm.add_constant(D), W, D.reshape(-1, 1) * W])).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,0.7006,0.176,3.971,0.000,0.355,1.046
x1,-1.1414,0.386,-2.957,0.003,-1.898,-0.385
x2,-0.9497,0.111,-8.582,0.000,-1.167,-0.733
x3,2.3274,0.250,9.291,0.000,1.836,2.818


# What if we have too many covariates

In [50]:
np.random.seed(123)
n = 100
d = 80
p = .2
W, D, Y = gen_data(n, d, p)

In [51]:
sm.OLS(Y, sm.add_constant(D)).fit(cov_type='HC0').summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,0.2740,0.141,1.946,0.052,-0.002,0.550
x1,0.6005,0.320,1.876,0.061,-0.027,1.228


Using all covariates, gives artificially small standard error due to overfitting.

In [52]:
W = W - W.mean(axis=0)
sm.OLS(Y, np.hstack([sm.add_constant(D), W, D.reshape(-1, 1) * W])).fit(cov_type='HC0').summary().tables[1]

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 161, but rank is 99
  warnings.warn('covariance of constraints does not have full '


,coef,std err,z,P>|z|,[0.025,0.975]
const,0.3128,2.58e-15,1.21e+14,0.000,0.313,0.313
x1,0.0398,3.35e-15,1.19e+13,0.000,0.040,0.040
x2,-0.9660,5.2e-15,-1.86e+14,0.000,-0.966,-0.966
x3,0.6915,4.09e-15,1.69e+14,0.000,0.691,0.691
x4,-0.6126,9.84e-15,-6.23e+13,0.000,-0.613,-0.613
x5,-0.1043,4.78e-15,-2.18e+13,0.000,-0.104,-0.104
x6,0.1444,7e-15,2.06e+13,0.000,0.144,0.144
x7,-0.1513,9.07e-15,-1.67e+13,0.000,-0.151,-0.151
x8,0.2437,3.76e-15,6.49e+13,0.000,0.244,0.244
x9,-0.2266,3.86e-15,-5.87e+13,0.000,-0.227,-0.227


We could try using a regularized linear regression like the Lasso.

In [53]:
from sklearn.linear_model import LassoCV

LassoCV().fit(np.hstack([D.reshape(-1, 1), W, D.reshape(-1, 1) * W]), Y).coef_[0]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.442e-02, tolerance: 1.240e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.319e-02, tolerance: 1.240e-02
  model = cd_fast.enet_coordinate_descent(


0.0

However, we see that Lasso returns a result that is very close to zero, due to regularization! So that doesn't solve our problem. How can we use the lasso more cleverly, while still getting the correct result, while controlling for many covariates?